## **Segmenting and Clustering Neighbourhoods in Toronto**
### All the 3 tasks are implemented in the same notebook.

In [2]:
import pandas as pd
import numpy as np

### Task #1

In [9]:
# Load HTML from Wikipedia
html_tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Print number of tables detected in HTML
print('Info: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
print(f'Number of tables detected: {len(html_tables)}')

# Print first rows of tables 
print('\nScraped tables:')
for i in range(len(html_tables)):
    print(html_tables[i].head())

Info: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Number of tables detected: 3

Scraped tables:
  Postal Code           Borough              Neighbourhood
0         M1A      Not assigned               Not assigned
1         M2A      Not assigned               Not assigned
2         M3A        North York                  Parkwoods
3         M4A        North York           Victoria Village
4         M5A  Downtown Toronto  Regent Park, Harbourfront
                                                  0   \
0                                                NaN   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NL   
3                                                  A   

                                                  1   \
0                              Canadian postal codes   
1  NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...   
2                                                 NS   
3                        

In [10]:
# Create dataframe from table containing postal codes

df = html_tables[0]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [11]:
# Rename columns: 
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.rename(columns={'Postal Code':'PostalCode',
                   'Neighbourhood':'Neighborhood'},
          inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PostalCode    180 non-null    object
 1   Borough       180 non-null    object
 2   Neighborhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [12]:
# Clean dataframe:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
#df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
df = df[df['Borough']!='Not assigned']

# More than one neighborhood can exist in one postal code area. 
# For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df = df.groupby(['PostalCode','Borough'],sort=False).agg(', '.join)
df.reset_index(inplace=True)

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
condition = df['Neighborhood']=='Not assigned'
df['Neighborhood'] = np.where(condition, df['Borough'], df['Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [13]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

### Task #2

In [15]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head(20)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [20]:
df.dtypes

PostalCode      object
Borough         object
Neighborhood    object
dtype: object

In [21]:
geo_df.dtypes

PostalCode     object
Latitude      float64
Longitude     float64
dtype: object

In [24]:
neighborhoods = pd.merge(df, geo_df, on='PostalCode')
neighborhoods.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
